# 1. Overview

In [44]:
import re
import time
import logging

import numpy as np
import pandas as pd

import nltk
from nltk import data
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

from bs4 import BeautifulSoup

import gensim
from gensim.models import Word2Vec

First, load the dataset. We use pandas to read the tab delimited file.

In [2]:
""""
header=0: indicates that the first line of the file contains column names
"delimiter=\t": indicates that the fields are separated by tabs
quoting=3: tells Python to ignore doubled quotes
"""

labeled_train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test_data = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)

In [3]:
labeled_train.columns

Index(['id', 'sentiment', 'review'], dtype='object')

As a remainder, the following code shows the first review given by a user. From the review, we could see HTML tags, abbreviations, punctuations. We need to clean the data before putting it to a machine learning model.

In [4]:
labeled_train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

# 2. Data Cleaning

In this part, we are going to do the text data preprocessing, which mainly includes deleting the punctuations, removing HTML tags, removing non-letters, etc. The following function shows how to finish these data cleaning steps.

In [32]:
def review_to_words(raw_review, remove_stopwords = False ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    try:
        review_text = BeautifulSoup(raw_review).get_text() 
    except TypeError:
        review_text = "None"
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    if remove_stopwords:
        stops = set(stopwords.words("english"))                  
        # 
        # 5. Remove stop words
        words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return words

In [33]:
clean_review = review_to_words( labeled_train["review"][0] )
print(clean_review)

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', 's', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'm', 'kay', 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', 'about', 

f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Now we could run through all the reviews.

In [34]:
# Get the number of reviews based on the dataframe column size
num_reviews = labeled_train["review"].size

# Initialize an empty list to hold the clean reviews
clean_labeled_train_reviews = []

print('Cleaning the labeled training set movie reviews...')

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_labeled_train_reviews.append( review_to_words( labeled_train["review"][i] ) )

Cleaning the labeled training set movie reviews...


f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [35]:
# Get the number of reviews based on the dataframe column size
num_reviews = unlabeled_train["review"].size

# Initialize an empty list to hold the clean reviews
clean_unlabeled_train_reviews = []

print('Cleaning the unlabeled training set movie reviews...')

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 


for i in range(num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_unlabeled_train_reviews.append( review_to_words( unlabeled_train["review"][i] ) )

Cleaning the labeled training set movie reviews...


f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Review 1000 of 115764

Review 2000 of 115764

Review 3000 of 115764

Review 4000 of 115764

Review 5000 of 115764

Review 6000 of 115764

Review 7000 of 115764

Review 8000 of 115764

Review 9000 of 115764

Review 10000 of 115764

Review 11000 of 115764

Review 12000 of 115764

Review 13000 of 115764

Review 14000 of 115764

Review 15000 of 115764

Review 16000 of 115764

Review 17000 of 115764

Review 18000 of 115764

Review 19000 of 115764

Review 20000 of 115764

Review 21000 of 115764

Review 22000 of 115764

Review 23000 of 115764

Review 24000 of 115764

Review 25000 of 115764

Review 26000 of 115764

Review 27000 of 115764

Review 28000 of 115764

Review 29000 of 115764

Review 30000 of 115764

Review 31000 of 115764

Review 32000 of 115764

Review 33000 of 115764

Review 34000 of 115764

Review 35000 of 115764

Review 36000 of 115764

Review 37000 of 115764

Review 38000 of 115764

Review 39000 of 115764

Review 40000 of 115764

Review 41000 of 115764

Review 42000 of 115764

R

In [67]:
# Get the number of reviews based on the dataframe column size
num_reviews = test_data["review"].size

# Initialize an empty list to hold the clean reviews
clean_test_data_reviews = []

print('Cleaning the test dataset movie reviews...')

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 

for i in range(num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_test_data_reviews.append( review_to_words( test_data["review"][i] ) )

Cleaning the test dataset movie reviews...


f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



# 3. Construct the Model

In this tutorial, to do the sentiment classification task, we want to first use word embedding to generate the representation of each word in a sentence.

In [42]:
sentences = []

print('Parsing the sentences from the labeled training set....')
start = time.time()

for review in clean_labeled_train_reviews:
    sentences.append(review)
    
print('Parsing the sentences from the unlabeled training set....')

for review in clean_unlabeled_train_reviews:
    sentences.append(review)
    
end = time.time()
duration = end - start

print("Time taken for getting all the text: ", duration, ' seconds')

Parsing the sentences from the labeled training set....
Parsing the sentences from the unlabeled training set....
Time taken for getting all the text:  0.06516218185424805  seconds


Since we have got all the words in the labeled and unlabeled dataset, we could use the word2vec model in [gensim](https://radimrehurek.com/gensim/models/word2vec.html) to generate the word vectors.

In [46]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
print("Training model...")
start = time.time()
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

end = time.time()
duration = end - start

print("Total time for training the word vectors: ", duration, " seconds")

2018-03-10 16:59:34,479 : INFO : collecting all words and their counts
2018-03-10 16:59:34,482 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2018-03-10 16:59:35,044 : INFO : PROGRESS: at sentence #10000, processed 2385575 words, keeping 51527 word types
2018-03-10 16:59:35,577 : INFO : PROGRESS: at sentence #20000, processed 4747504 words, keeping 67813 word types
2018-03-10 16:59:36,114 : INFO : PROGRESS: at sentence #30000, processed 7109778 words, keeping 81728 word types
2018-03-10 16:59:36,657 : INFO : PROGRESS: at sentence #40000, processed 9467206 words, keeping 92655 word types
2018-03-10 16:59:37,236 : INFO : PROGRESS: at sentence #50000, processed 11825236 words, keeping 99583 word types
2018-03-10 16:59:37,801 : INFO : PROGRESS: at sentence #60000, processed 14190901 words, keeping 105913 word types
2018-03-10 16:59:38,365 : INFO : PROGRESS: at sentence #70000, processed 16573274 words, keeping 111433 word types
2018-03-10 16:59:38,924 : INFO : PROGRESS: at sentence #80000, processed 18978383 words, keeping 116764 word types
2018-03-10 16:59:39,483 : INFO : PROGRESS: at sentence #90000, processed 21360809 words, 

2018-03-10 17:00:39,762 : INFO : PROGRESS: at 23.39% examples, 502530 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:00:40,777 : INFO : PROGRESS: at 23.77% examples, 501627 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:00:41,785 : INFO : PROGRESS: at 24.13% examples, 500436 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:00:42,811 : INFO : PROGRESS: at 24.48% examples, 498923 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:00:43,822 : INFO : PROGRESS: at 24.82% examples, 497458 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:00:44,826 : INFO : PROGRESS: at 25.14% examples, 495634 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:00:45,838 : INFO : PROGRESS: at 25.54% examples, 495300 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:00:46,843 : INFO : PROGRESS: at 25.92% examples, 494678 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:00:47,865 : INFO : PROGRESS: at 26.33% examples, 494404 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:00:48,884 : INFO : PROGRESS: at 26.76% examples, 494703 wor

2018-03-10 17:02:00,464 : INFO : PROGRESS: at 57.72% examples, 511607 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:02:01,470 : INFO : PROGRESS: at 58.16% examples, 511761 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:02:02,479 : INFO : PROGRESS: at 58.60% examples, 511911 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:02:03,486 : INFO : PROGRESS: at 59.04% examples, 512071 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:02:04,499 : INFO : PROGRESS: at 59.49% examples, 512240 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:02:05,505 : INFO : PROGRESS: at 59.93% examples, 512387 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:02:06,509 : INFO : PROGRESS: at 60.37% examples, 512552 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:02:07,518 : INFO : PROGRESS: at 60.80% examples, 512683 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:02:08,543 : INFO : PROGRESS: at 61.25% examples, 512805 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:02:09,564 : INFO : PROGRESS: at 61.69% examples, 512891 wor

2018-03-10 17:03:21,357 : INFO : PROGRESS: at 91.72% examples, 511626 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:03:22,358 : INFO : PROGRESS: at 92.13% examples, 511543 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:03:23,382 : INFO : PROGRESS: at 92.55% examples, 511505 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:03:24,402 : INFO : PROGRESS: at 92.93% examples, 511286 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:03:25,420 : INFO : PROGRESS: at 93.33% examples, 511102 words/s, in_qsize 8, out_qsize 1
2018-03-10 17:03:26,430 : INFO : PROGRESS: at 93.74% examples, 511099 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:03:27,435 : INFO : PROGRESS: at 94.12% examples, 510880 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:03:28,446 : INFO : PROGRESS: at 94.45% examples, 510394 words/s, in_qsize 8, out_qsize 0
2018-03-10 17:03:29,453 : INFO : PROGRESS: at 94.88% examples, 510410 words/s, in_qsize 7, out_qsize 0
2018-03-10 17:03:30,462 : INFO : PROGRESS: at 95.26% examples, 510189 wor

Total time for training the word vectors:  252.76197171211243  seconds


We have got the word vector model. To load this model, you could use the following code:

```Python
model = Word2Vec.load("300features_40minwords_10context")
```

In [65]:
def makeFeatureVec(sentence, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((300,),dtype="float32")
    #
    nwords = 0.
    #
    # Loop over each word in the review
    for word in sentence: 
        nwords = nwords + 1.
        featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [66]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter % 1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [70]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
print("Creating average feature vecs for training reviews")

trainDataVecs = getAvgFeatureVecs(clean_labeled_train_reviews, model, 300)

print("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for training reviews
Review 0 of 25000


KeyError: "word 'wiz' not in vocabulary"

In [71]:
clean_labeled_train_reviews[0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',